# Reading data

In [67]:
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import argparse

In [68]:
def file_len(filename):
    with open(filename) as f:
        for i,l in enumerate(f):
            pass
    return i+1

In [69]:
def read_csv_format_file(filename_queue):
    reader=tf.TextLineReader()
    _,record_string=reader.read(filename_queue)
    record_defaults=[[1],[1],[1],[1],[1]]
    col1,col2,col3,col4,label=tf.decode_csv(record_string,record_defaults=record_defaults)
    features=tf.pack([col1,col2,col3,col4])
    return features,label

In [70]:
def input_pipeline(filenames,batch_size,num_epochs=None):
    filename_queue=tf.train.string_input_producer(filenames,num_epochs=num_epochs,shuffle=True)
    example,label=read_csv_format_file(filename_queue)
    
    min_after_dequeue=10000
    capacity=min_after_dequeue+3*batch_size
    example_batch,label_batch=tf.train.shuffle_batch(
    [example,label],batch_size=batch_size,capacity=capacity,
    min_after_dequeue=min_after_dequeue)
    return example_batch,label_batch

In [73]:
filenames=['file0.csv']
file_length=file_len(filenames[0]) # only first file length
examples,labels=input_pipeline(filenames,batch_size=file_length,num_epochs=1)

In [77]:
with tf.Session() as sess:
    init_op = tf.group(tf.initialize_all_variables(),tf.initialize_local_variables())
    sess.run(init_op)
    
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(coord=coord)
    
    try:
        while not coord.should_stop():
            example_batch,label_batch=sess.run([examples,labels])
            print(example_batch,label_batch)
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)

[[5 5 5 5]
 [3 3 3 3]
 [1 1 1 1]
 [4 4 4 4]
 [2 2 2 2]] [5 3 1 4 2]
Done training -- epoch limit reached
